# NDN Simulation - Google Colab Setup

This notebook runs the NDN simulation with free GPU acceleration.

## Steps:
1. **Enable GPU**: Runtime → Change runtime type → GPU (T4 free or V100/A100 Pro)
2. **Run all cells below** in order
3. **Check results** in the output

## Repository:
- GitHub: https://github.com/tsaipraveen99/ndn-nwsim-Sai_work


In [ ]:
# Step 1: Clone Repository from GitHub
!git clone https://github.com/tsaipraveen99/ndn-nwsim-Sai_work.git
%cd ndn-nwsim-Sai_work

# Verify files
!ls -la *.py | head -10


In [ ]:
# Step 2: Install Dependencies
%pip install networkx numpy torch scipy matplotlib pandas scikit-learn dill bitarray hdbscan tensorflow mmh3


In [ ]:
# Step 3: Verify GPU
import torch
print("="*60)
print("GPU VERIFICATION")
print("="*60)
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("✅ GPU ready for DQN training!")
else:
    print("❌ No GPU - check Runtime → Change runtime type → GPU")


In [ ]:
# Step 4: Configure Simulation
import os

# Enable DQN to use GPU
os.environ['NDN_SIM_USE_DQN'] = '1'

# Network Configuration
os.environ['NDN_SIM_NODES'] = '200'              # Core routers
os.environ['NDN_SIM_PRODUCERS'] = '40'           # 20% of nodes
os.environ['NDN_SIM_USERS'] = '500'              # 2.5x nodes
os.environ['NDN_SIM_CONTENTS'] = '2000'          # 10x nodes

# Simulation Parameters
os.environ['NDN_SIM_ROUNDS'] = '50'              # Evaluation rounds
os.environ['NDN_SIM_REQUESTS'] = '30'            # Requests per round
os.environ['NDN_SIM_WARMUP_ROUNDS'] = '5'        # Warm-up rounds

# Cache Configuration
os.environ['NDN_SIM_CACHE_CAPACITY'] = '10'      # 0.5% of catalog
os.environ['NDN_SIM_ZIPF_PARAM'] = '0.8'         # Heavy-tail distribution
os.environ['NDN_SIM_CACHE_POLICY'] = 'combined'

# Performance Settings
os.environ['NDN_SIM_QUIET'] = '1'                # Suppress verbose output
os.environ['NDN_SIM_SKIP_DELAYS'] = '1'          # Skip sleep delays

# Topology (optional)
os.environ['NDN_SIM_TOPOLOGY'] = 'watts_strogatz'  # or 'barabasi_albert'

print("Configuration:")
for key, value in os.environ.items():
    if key.startswith('NDN_SIM_'):
        print(f"  {key} = {value}")


In [ ]:
# Step 5: Run Benchmark
!python benchmark.py


In [ ]:
# Step 6: View Results
import json
from pathlib import Path

results_file = Path('benchmark_checkpoints/benchmark_results.json')
if results_file.exists():
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    print("\n" + "="*60)
    print("BENCHMARK RESULTS")
    print("="*60)
    for name, result in results.items():
        hit_rate = result.get('hit_rate', 0)
        std = result.get('hit_rate_std', 0)
        runs = result.get('num_runs', 0)
        print(f"{name:15s}: {hit_rate:6.2f}% ± {std:.2f}% ({runs} runs)")
else:
    print("Results file not found. Check output above for errors.")


In [ ]:
# Step 7: Download Results
from google.colab import files

# Download results JSON
if Path('benchmark_checkpoints/benchmark_results.json').exists():
    files.download('benchmark_checkpoints/benchmark_results.json')
    print("✅ Results downloaded!")
else:
    print("❌ Results file not found")


## Step 8: Update Code (When You Push Changes to GitHub)

Run this cell whenever you push updates to GitHub to get the latest code.


In [ ]:
# Run this cell whenever you push updates to GitHub
%cd ndn-nwsim-Sai_work
!git pull
print("✅ Code updated from GitHub!")


In [ ]:
# Run simulation with GPU acceleration
!python main.py


In [ ]:
# Check results
!tail -50 logs/simulation_results.log 2>/dev/null || echo "Results file not found"
